<a href="https://colab.research.google.com/github/Jorgeduardo13/secop-analysis/blob/master/Final_Project_Team_82_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1.Importe de Archivos SECOP I

In [ ]:
import sys
!pip install sodapy
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension
!pip uninstall pandas_profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [ ]:
from sodapy import Socrata
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import plotly.express as px
import datetime
from pathlib import Path
from ipywidgets import widgets
from pandas_profiling import ProfileReport
from datetime import datetime


In [ ]:
def Nombre_Limpio(text):
  words=text.split("_")
  s= " ".join(words)
  return s
  
def Outliers(df,cols):
  df_copy=df.copy()
  for col in cols:
    ValMax=df_copy[col].quantile(0.75)+1.5*(df_copy[col].quantile(0.75)-df_copy[col].quantile(0.25))
    df_copy=df_copy[df_copy[col]<ValMax].reset_index(drop=True)
  return df_copy

def GraficosConteos(df,w):
  Prueba=df[w].value_counts().reset_index()
  Prueba.columns=[w,"Cantidad de registros"]
  Prueba=Prueba.sort_values(by="Cantidad de registros",ascending=False)
  fig=px.bar(Prueba[:20],x=w,y="Cantidad de registros",text="Cantidad de registros", title=f"Gráfico de conteo {w}")
  fig.show()
def GraficosBoxplot(df,w):
  fig=px.box(df,y=w, title=f"Gráfico de boxplot {w}")
  fig.show()



In [ ]:
client = Socrata("www.datos.gov.co", None)

results = client.get("xvdy-vvsk",limit=50000)

df_secopi = pd.DataFrame.from_records(results)
df_secopi.reset_index(drop=True)
df_secopi.columns

In [ ]:
ProfileReport(df_secopi, title="Pre analisis Secop I").to_file("PreAnalisis Secop I.html")

** Hay que cambiar la naturaleza de las variables. Todas salieron objeto debido a que son sacadas de un API

1. Hay datos desde el 2011 o 2010
2. Hay una gran cantidad de contratos que no tienen un año confirmado del contrato lo tienen catalogado como 0.
3. Existe una gran concentración (80%) de los contratos en territorial y un 19% en nacional. Sugiero quitar los que no estan definidos (0.1%)
4. La entidad de territorial distrital municipal se divide en 6 niveles donde el lvl 6 es el que más tiene contratos(19%) seguido por el 5(2.8%) y el 4(2.3%).
5. No creo que el nit de la entidad o el nombre especifico de esta sirva como variable a analizar por la alta cardinalidad. Existen 2646 entidades unicas en 10k de datos, sin embargo hay menos nits que nombres de empresa. Esto quiere decir que hay empresas no registradas con NIT.
6. Existe una concentracion en los procesos 12, 4 y 13 de casi el 93.6%. Contratacion Directa, Regin Especial y Contratación de minima cuantia son los tipos de procesos que sobre salen.
7. Más de la mitad de los contratos (59.4%) son conratos celebrados. Mientras que el 26.6% de los contratos son liquidados. Solo el 7.7% de los contratos estan en convocatoria. El otro 6% Son contratos que no hay finalizado.
8. Prestación de servicios se divide practicamente en 2 los que no tienen ninguno o fueron presentados por profesionales. 
9. El regimen contractual es principalmente de Estatuto General de contratación , id=3. (71.8%) o de Regimen Especial id=2 (28%)
10. El objeto contractual que tienen mayor cantidad de registros es de Gestión Pública(41.7%) y Servicios de Salud(11.2%).





# 2.Importe de Archivos SECOP II




In [ ]:
!pip install sodapy
import pandas as pd
from sodapy import Socrata


client = Socrata("www.datos.gov.co", None)


results = client.get("jbjy-vk9h", limit=50000)

df_secopii = pd.DataFrame.from_records(results)

In [ ]:
ProfileReport(df_secopii, title="Pre analisis Secop II").to_file("PreAnalisis Secop II.html")

# 3.Tienda Virtual

In [ ]:
client = Socrata("www.datos.gov.co", None)


results = client.get("rgxm-mmea", limit=20000)

df_tienda = pd.DataFrame.from_records(results)

In [ ]:
df_tienda

In [ ]:
ProfileReport(df_tienda, title="Pre analisis Tienda Virtual").to_file("PreAnalisis Tienda Virtual.html")

# Cleaning Data

## Select columns to work


Secop 1

In [ ]:
cols_nums1= ["cuantia_proceso","tiempo_adiciones_en_dias","cuantia_contrato","valor_total_de_adiciones","valor_contrato_con_adiciones","plazo_de_ejec_del_contrato"]
no_cols= ['anno_cargue_secop','nit_de_la_entidad',"id_tipo_de_proceso","id_regimen_de_contratacion","id_objeto_a_contratar","fecha_de_cargue_en_el_secop",
           'numero_de_constancia','id_grupo','id_familia','id_clase','id_ajudicacion','tipo_doc_representante_legal',
           'identific_del_represen_legal', 'nombre_del_represen_legal','rango_de_ejec_del_contrato','compromiso_presupuestal','id_origen_de_los_recursos',
           'proponentes_seleccionados','calificacion_definitiva','id_sub_unidad_ejecutora','nombre_sub_unidad_ejecutora',
           'espostconflicto','marcacion_adiciones','posicion_rubro', 'nombre_rubro', 'valor_rubro','sexo_replegal_entidad', 'pilar_acuerdo_paz', 
           'punto_acuerdo_paz','departamento_entidad', 'ultima_actualizacion','numero_de_proceso','numero_del_contrato',
          'tiempo_adiciones_en_meses','origen_de_los_recursos','codigo_bpin',	'ruta_proceso_en_secop_i','moneda','nivel_entidad','orden_entidad','objeto_del_contrato_a_la'	]
fechas_secop1= [ 'fecha_de_firma_del_contrato', 'fecha_ini_ejec_contrato','fecha_fin_ejec_contrato']

##Identify null values in columns

In [ ]:
df_secopi[fechas_secop1].isna().any()

In [ ]:
null_columns=df_secopi.columns[df_secopi.isnull().any()]
df_secopi[null_columns].isnull().sum()/len(df_secopi)

In [ ]:
#Impute values
clean_secop1=df_secopi.drop(columns=no_cols,axis=1)
clean_secop1=clean_secop1.dropna()


In [ ]:
# Cast variables to acording to their nature
for x in cols_nums1:
  clean_secop1[x]=clean_secop1[x].apply(lambda y: float(y))
for x in fechas_secop1:
  clean_secop1[x]=clean_secop1[x].apply(lambda y: datetime.strptime(str(y),"%m/%d/%Y"))

In [ ]:
#Lower main variable
clean_secop1.detalle_del_objeto_a_contratar=clean_secop1.detalle_del_objeto_a_contratar.apply(lambda x: x.lower())


In [ ]:
# Cambio Formato de los titulos 
cols_dirty_secopi=clean_secop1.columns.to_list()
cols_clean_secopi=list (map(Nombre_Limpio,cols_dirty_secopi))
clean_secop1.columns=cols_clean_secopi
cols_nums1=list (map(Nombre_Limpio,cols_nums1))
col_dates1 = list (map(Nombre_Limpio,fechas_secop1))
col_dates1

In [ ]:
# Identify Outliers, days are too concentrated at the right that's why the filters is different for this variable.
cleani_sindias=Outliers(clean_secop1,["cuantia proceso","cuantia contrato",'valor contrato con adiciones','plazo de ejec del contrato'])
cleani=cleani_sindias[cleani_sindias['tiempo adiciones en dias']<cleani_sindias['tiempo adiciones en dias'].quantile(0.99)].reset_index(drop=True)
cleani

In [ ]:
#Select columns string variables to graph
colsi=clean_secop1.columns.to_list()
colsi=[x for x in colsi[1:] if x not in col_dates1 and x not in cols_nums1 and x not in ['c digo de la entidad','identificacion del contratista']]
colsi

In [ ]:
#Graph!
for col in colsi:
  GraficosConteos(cleani,col)

In [ ]:
#Graph!
for col in cols_nums1:
  GraficosBoxplot(cleani,col)

In [ ]:

cols_no_ii=['nit_entidad','orden','referencia_del_contrato','condiciones_de_entrega','tipodocproveedor','obligaci_n_ambiental','valor_de_pago_adelantado','valor_pendiente_de_pago',
            'valor_amortizado','c_digo_bpin', 'anno_bpin','urlproceso','puntos_del_acuerdo','pilares_del_acuerdo','proceso_de_compra','id_contrato','codigo_de_categoria_principal',
            'justificacion_modalidad_de','documento_proveedor','habilita_pago_adelantado'	,'liquidaci_n'	,'obligaciones_postconsumo','reversion','valor_pendiente_de','saldo_vigencia',
            'espostconflicto','estado_bpin']
cols_nums2=['valor_del_contrato','valor_del_contrato','valor_facturado', 'valor_pagado', 'valor_pendiente_de_ejecucion','saldo_cdp','dias_adicionados']
cols_date2=["fecha_de_firma" ,'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato','fecha_de_inicio_de_ejecucion', 'fecha_de_fin_de_ejecucion']



In [ ]:
clean_secop2=df_secopii.drop(columns=cols_no_ii)
clean_secop2=clean_secop2.dropna()

In [ ]:
# Cast variables to acording to their nature
for x in cols_nums2:
  clean_secop2[x]=clean_secop2[x].apply(lambda y: float(y))
for x in cols_date2:
  clean_secop2[x]=clean_secop2[x].apply(lambda y: datetime.strptime(str(y)[:10],"%Y-%m-%d"))
clean_secop2.descripcion_del_proceso	=clean_secop2.descripcion_del_proceso.apply(lambda x: x.lower())

In [ ]:
#Cleaning tittles of columns
cols_dirty_secopii=clean_secop2.columns.to_list()
cols_clean_secopii=list (map(Nombre_Limpio,cols_dirty_secopii))
clean_secop2.columns=cols_clean_secopii
cols_nums2=list (map(Nombre_Limpio,cols_nums2))
col_dates2 = list (map(Nombre_Limpio,cols_date2))
col_dates2

In [ ]:
# Identify Outliers, days are too concentrated at the right that's why the filters is different for this variable.
cleanii_sindias=Outliers(clean_secop2,['valor del contrato','valor facturado', 'valor pagado', 'valor pendiente de ejecucion','saldo cdp'])
cleanii=cleanii_sindias[cleanii_sindias['dias adicionados']<cleanii_sindias['dias adicionados'].quantile(0.99)].reset_index(drop=True)
cleanii


In [ ]:
#Select columns string variables to graph
colsii=clean_secop1.columns.to_list()
colsii=[x for x in cols_clean_secopii if x not in col_dates2 and x not in cols_nums2]
colsii

In [ ]:
for col in colsii:
  GraficosConteos(cleanii,col)

In [ ]:
for col in cols_nums2:
  GraficosBoxplot(cleanii,col)

Tienda Virtual

In [ ]:
clean_tienda=df_tienda.drop(columns=['espostconflicto','nit_proveedor'])
cols_nums3=["a_o","total"]
fechas_tienda=["fecha"]
for x in cols_nums3:
  clean_tienda[x]=clean_tienda[x].apply(lambda y: float(y))
for x in fechas_tienda:
  clean_tienda[x]=clean_tienda[x].apply(lambda y: datetime.strptime(str(y[:10]),"%Y-%m-%d"))

Graficas de conteo por variable

# 4.Información Geográfica

In [ ]:
!sudo apt install libspatialindex-dev
!pip install geopandas
!pip install descartes
!pip install geopandas matplotlib

In [ ]:
!pip install geopandas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import IPython.display as display
import ogr                           # GDAL/OGR
import shapely                       # Herramientas para manipulación geométrica
from shapely.geometry import *

%matplotlib inline

In [ ]:
#Mun_Col = gpd.read_file('Municipios_Col2.json')

In [ ]:
#Mun_Col.plot(column='dpto_cnmbr', 
#             cmap='viridis', 
#             figsize=(20, 13))

In [ ]:
#Mun_Col.head()